In [7]:
# Dynamic Pricing for Urban Parking Lots
# Libraries
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from bokeh.plotting import figure, show, output_notebook
output_notebook()

# Load Data
df = pd.read_csv("dataset.csv")

# Preprocess
df["timestamp"] = pd.to_datetime(df["LastUpdatedDate"].astype(str) + " " + df["LastUpdatedTime"].astype(str), dayfirst=True, errors="coerce")
df.rename(columns={
    "ID": "parking_lot_id",
    "Capacity": "capacity",
    "Occupancy": "occupancy",
    "QueueLength": "queue_length",
    "TrafficConditionNearby": "traffic",
    "IsSpecialDay": "special_day",
    "VehicleType": "vehicle_type",
    "Latitude": "latitude",
    "Longitude": "longitude"
}, inplace=True)

# Convert types
cols_to_convert = ["capacity", "occupancy", "queue_length", "traffic", "special_day"]
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Constants
BASE_PRICE = 10
ALPHA = 2
LAMBDA = 0.1
PRICE_MIN = 5
PRICE_MAX = 20

# Helper: Normalize
def normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-6)

# Helper: Vehicle Type Weights
def get_vehicle_weight(vehicle):
    try:
        return {"car": 1, "bike": 0.5, "truck": 1.5}.get(str(vehicle).lower(), 1)
    except:
        return 1

# Haversine Formula
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 6371 * 2 * asin(sqrt(a)) * 1000  # meters

# Model 1: Linear Pricing
def model1_linear_pricing(row, prev_price):
    occ_ratio = row["occupancy"] / row["capacity"]
    new_price = prev_price + ALPHA * occ_ratio
    return np.clip(new_price, PRICE_MIN, PRICE_MAX)

# Model 2: Demand-Based Pricing
def model2_demand_pricing(row):
    occ = row["occupancy"] / row["capacity"]
    queue = row["queue_length"]
    traffic = row["traffic"]
    special = row["special_day"]
    vtype = get_vehicle_weight(row["vehicle_type"])
    demand = 0.5*occ + 0.2*queue - 0.1*traffic + 0.3*special + 0.4*vtype
    return np.clip(BASE_PRICE * (1 + LAMBDA * demand), PRICE_MIN, PRICE_MAX)

# Model 3: Competitive Pricing
def model3_competitive(row, df_subset):
    my_lat, my_lon = row["latitude"], row["longitude"]
    my_price = model2_demand_pricing(row)
    competitors = []
    for _, comp in df_subset.iterrows():
        if comp.name == row.name: continue
        dist = haversine(my_lon, my_lat, comp["longitude"], comp["latitude"])
        if dist < 500:
            comp_price = model2_demand_pricing(comp)
            competitors.append((comp_price, dist))
    if competitors:
        avg_comp_price = np.mean([c[0] for c in competitors])
        if row["occupancy"] >= row["capacity"] and my_price > avg_comp_price:
            my_price -= 1  # encourage rerouting
        elif my_price < avg_comp_price:
            my_price += 1  # capitalize on demand
    return np.clip(my_price, PRICE_MIN, PRICE_MAX)

# Apply Models
def simulate_prices(df):
    df = df.sort_values(by="timestamp")
    df["price_model1"] = BASE_PRICE
    df["price_model2"] = BASE_PRICE
    df["price_model3"] = BASE_PRICE
    last_prices = {}
    
    for t in sorted(df["timestamp"].dropna().unique()):
        batch = df[df["timestamp"] == t]
        for idx, row in batch.iterrows():
            lot_id = row["parking_lot_id"]
            prev_price = last_prices.get(lot_id, BASE_PRICE)
            df.at[idx, "price_model1"] = model1_linear_pricing(row, prev_price)
            df.at[idx, "price_model2"] = model2_demand_pricing(row)
            df.at[idx, "price_model3"] = model3_competitive(row, batch)
            last_prices[lot_id] = df.at[idx, "price_model1"]  # update model1 price
    return df

# Run Simulation
df = simulate_prices(df)

# Bokeh Plot
lot_id = df["parking_lot_id"].unique()[0]
plot_df = df[df["parking_lot_id"] == lot_id]
p = figure(title=f"Real-time Prices - Lot {lot_id}", x_range=plot_df["timestamp"].astype(str), height=300)
p.line(plot_df["timestamp"].astype(str), plot_df["price_model1"], legend_label="Model 1", color="blue")
p.line(plot_df["timestamp"].astype(str), plot_df["price_model2"], legend_label="Model 2", color="green")
p.line(plot_df["timestamp"].astype(str), plot_df["price_model3"], legend_label="Model 3", color="red")
p.xaxis.major_label_orientation = 1.2
p.legend.location = "top_left"
show(p)


Loading BokehJS ...

C:\Users\rbhar\AppData\Local\Temp\ipykernel_20040\826967336.py:105: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.211438474870018' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[idx, "price_model1"] = model1_linear_pricing(row, prev_price)
